# Sciunit Cross-OS Reproducibility Demo on Chameleon

1. Create an **Ubuntu VM** to author a Sciunit experiment
2. Create a **Fedora VM** to reproduce the experiment
3. Use SSH to interact with both machines
4. Demonstrate cross-OS reproducibility using Sciunit

This notebook focuses only on **infrastructure setup**.
Sciunit commands are executed directly on the VMs


In [ ]:
import chi, os, time, datetime
from chi import lease
from chi import server
from chi import context
from chi import network

context.version = "1.0"
context.choose_project()
context.choose_site(default="KVM@TACC")

username = os.getenv("USER")


In [ ]:
exp_name = "sciunit_cross_os_demo"

ubuntu_server_name = f"{exp_name}-ubuntu-{username}"
centOS_server_name = f"{exp_name}-centOS-{username}"
lease_name = f"{exp_name}-lease-{username}"

print("Experiment name:", exp_name)


In [ ]:
l = lease.Lease(
    lease_name,
    duration=datetime.timedelta(hours=6)
)

l.add_flavor_reservation(
    id=server.get_flavor_id("m1.small"),
    amount=2
)

l.submit(idempotent=True)
l.show()


In [ ]:
ubuntu_image = "CC-Ubuntu22.04"

ubuntu_vm = server.Server(
    name=ubuntu_server_name,
    image_name=ubuntu_image,
    flavor_name=l.get_reserved_flavors()[0].name
)

ubuntu_vm.submit(idempotent=True)


In [ ]:
centOS_image = "CC-CentOS8-Stream-20230531"

centOS_vm = server.Server(
    name=centOS_server_name,
    image_name=centOS_image,
    flavor_name=l.get_reserved_flavors()[0].name
)

centOS_vm.submit(idempotent=True)


In [ ]:
ubuntu_vm.associate_floating_ip()
ubuntu_ip = ubuntu_vm.get_floating_ip()
print("Ubuntu VM IP:", ubuntu_ip)

In [ ]:
centOS_vm.associate_floating_ip()
centOS_ip = centOS_vm.get_floating_ip()
print("centOS VM IP:", centOS_ip)

In [ ]:
sg_list = network.list_security_groups(name_filter="allow-ssh")

if sg_list:
    sg = sg_list[0]
else:
    sg = network.SecurityGroup({
        "name": "allow-ssh",
        "description": "Enable SSH access"
    })
    sg.add_rule("ingress", "tcp", 22)
    sg.submit()

ubuntu_vm.add_security_group(sg.id)
fedora_vm.add_security_group(sg.id)


In [ ]:
ubuntu_vm.check_connectivity()
centOS_vm.check_connectivity()

print("Both Ubuntu and centOS VMs are ready.")


In [ ]:
print("SSH into Ubuntu (Sciunit author):")
print(f"ssh cc@{ubuntu_ip}")

print("SSH into centOS (Sciunit reproducer ):")
print("\nSSH cc@{centOS_ip}")


## Install Sciunit on the Ubuntu VM

The following commands must be run **inside the Ubuntu VM terminal**, not in Jupyter.
Sciunit operates at the operating-system level and must be installed in a real shell
environment to correctly capture execution provenance.

### Step 1: Verify Python installation

```bash
python3 --version


### Step 2: Install pipx
pipx allows us to safely install command-line Python applications without modifying
the system Python environment.

sudo apt install -y pipx


### Step 3: Enable pipx in your shell

pipx ensurepath

source ~/.bashrc


### Step 4: Install Sciunit

pipx install sciunit2

once installed check the version with the following command

sciunit --version

expected output is sciunit2 0.4.post115.dev43584757

## Cleanup 

In [ ]:
ubuntu_vm.delete()
fedora_vm.delete()
l.delete()

print("Cleanup commands prepared (commented out).")


_______________________________________________________________________________